The following workbooks performs a bulk differential expression analysis for Teton cytoprofiling cell output. Rather than explicitly modeling the distribution of counts, this approach evaluates the changes in average counts across conditions, and assesses the change of each target relative to how much change is observed for most genes. This relies on two assumptions
* Most targets are not changing across the conditions
* Homoscedasticity of log-transformed counts

Briefly, the analysis below will
* Peform default filtering of cells (by area, assigned counts, and assigned rate)
* Normalize cells by assigned counts and samples by median of ratios
* Calculate log ratio for each targets
* Estimate standard deviation from MAD of log ratios across all targets
* Normalize each log ratio by standard deviation and report Z score

To use the workbook, update the values for hte following fields in the cell below
* cell_stats_file: path to cytoprofiling RawCellStats.parquet output (typically found in Cytoprofiling/Instrument/RawCellStats.parquet in the Teton output directory)
* batch_names: Batches to include in the analysis
* control : Label of well to use as control
* cases : Labels of wells to use as cases

In [ ]:
cell_stats_file = "/path/to/RawCellStats.parquet"
run_name = "run_name"

# RNA batches
batch_names = ["B02", "B03", "B04", "B05", "B06", "B07",]

#
# Well labels for control and cases
#
control = "control_label"
cases = ["case_label1", "case_label_2" ]

In [ ]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from cytoprofiling import get_wells, filter_cells, get_default_normalization_targets, normalize_cells_by_aggregated_counts, normalize_wells_by_median_of_ratios

#
# load cell table
#
df = pd.read_parquet(cell_stats_file)

#
# load the data
#
well2label = {}
for well in get_wells(df):
    well2label[well] = df["WellLabel"][df["Well"] == well].unique()[0]

#
# filter cells
#
df = filter_cells(df, batch_names=batch_names, stats={})


z_dfs = []
for case in cases:
    dir = f"{run_name}/{case}_vs_{control}"
    os.makedirs(dir, exist_ok=True)

    #
    # filter to target wells
    #
    control_wells = []
    case_wells = []
    target_wells = []
    for well in well2label:
        if control == well2label[well]:
            control_wells.append(well)
        if case == well2label[well]:
            case_wells.append(well)
    target_wells = control_wells + case_wells


    case_df = df.loc[df["Well"].isin(target_wells)]
    case_df.reset_index(drop=True, inplace=True)

    norm_df = normalize_cells_by_aggregated_counts(case_df, batch_names=batch_names)
    well_names = get_wells(norm_df)
    norm_df = normalize_wells_by_median_of_ratios(norm_df, batch_names = batch_names, well_names = well_names)

    all_targets = []
    all_zs = []

    #
    # Calculate SD estimate from MAD
    #
    shared_sds = []
    for batch_name in batch_names:
        targets = get_default_normalization_targets(norm_df, batch_name)
        control_values = []
        case_values = []
        for target in targets:
            control_values.append(np.log2(np.nanmean(norm_df.loc[norm_df["Well"].isin(control_wells), target])))
            case_values.append(np.log2(np.nanmean(norm_df.loc[norm_df["Well"].isin(case_wells), target])))
        
        shared_sd = 1.4826 * np.nanmedian(np.abs(np.array(control_values) - np.array(case_values)))
        shared_sds.append(shared_sd)
    shared_sd = np.nanmedian(shared_sds)

    min_value = float("nan")
    max_value = float("nan")

    for batch_name in batch_names:
        targets = get_default_normalization_targets(norm_df, batch_name)
        control_values = []
        case_values = []
        for target in targets:
            control_values.append(np.log2(np.nanmean(norm_df.loc[norm_df["Well"].isin(control_wells), target])))
            case_values.append(np.log2(np.nanmean(norm_df.loc[norm_df["Well"].isin(case_wells), target])))
      
        min_value = np.nanmin(list(control_values) + list(case_values) + [min_value,])
        max_value = np.nanmax(list(control_values) + list(case_values) + [max_value,])
        
        plt.scatter(control_values, case_values, label=batch_name)

        all_targets.extend(targets)
        all_zs.extend((np.array(case_values) - np.array(control_values))/shared_sd)    
    
    #
    # Add identity line
    #
    plt.plot([min_value, max_value], [min_value, max_value], color="black")

    #
    # Added dotted lines to plot for ~3 standard deviations
    #
    plt.plot([min_value + 3 * shared_sd, max_value + 3 * shared_sd], [min_value, max_value ], color = "red", linestyle="--") 
    plt.plot([min_value - 3 * shared_sd, max_value - 3 * shared_sd], [min_value, max_value ], color = "red", linestyle="--")

    
    plt.title(f"{run_name}")
    plt.xlabel(f"log2 {control}")
    plt.ylabel(f"log2 {case}")
    plt.legend()
    plt.savefig(f"{dir}/{control}_vs_{case}.png")
    plt.show()
    plt.close()


    z_df = pd.DataFrame.from_dict({"Target" : all_targets, f"Z_{case}" : all_zs})
    z_dfs.append(z_df)

#
# Output merged table of empirical Z values
#
z_df = z_dfs[0]
for other_df in z_dfs[1:]:
    z_df = z_df.merge(other_df, on="Target")
z_df.to_csv(f"{run_name}/z_values.csv", index=False)